# Created by Jacek Grzybowski
## second attempt to make more accurate classification

## Part 1

In [56]:
# import libraries
import cv2
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Activation, MaxPooling2D, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

In [57]:
def load_img(indir, exclude = []):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir): 
            continue
        the_class = class_dir
        if the_class in exclude:
            continue
        print("Loading:",class_dir)
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('weather', ["rain", "shine"])
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels

Loading: cloudy
Loading: sunrise
loaded 657  samples
classes {'cloudy', 'sunrise'}


In [58]:
# create end train/test dataset

labels_vec = LabelEncoder().fit_transform(org_labels)
one_hot_labels = keras.utils.to_categorical(labels_vec)
# org_samples, one_hot_labels = shuffle(org_samples, one_hot_labels)

X_train, X_test, y_train, y_test = train_test_split(org_samples, one_hot_labels, test_size=0.25, random_state=666)

print(X_train.shape)
print(y_train.shape)


(492, 64, 64, 3)
(492, 2)


In [59]:
# create and compie model
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [60]:
# train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=16, batch_size=16)

Epoch 1/16
31/31 [==============================] - 2s 56ms/step - loss: 0.9285 - accuracy: 0.8699 - val_loss: 0.7509 - val_accuracy: 0.9576
Epoch 2/16
31/31 [==============================] - 2s 70ms/step - loss: 0.1882 - accuracy: 0.9654 - val_loss: 0.9796 - val_accuracy: 0.9091
Epoch 3/16
31/31 [==============================] - 1s 38ms/step - loss: 0.1067 - accuracy: 0.9776 - val_loss: 0.0261 - val_accuracy: 0.9939
Epoch 4/16
31/31 [==============================] - 1s 45ms/step - loss: 0.0551 - accuracy: 0.9858 - val_loss: 0.0759 - val_accuracy: 0.9758
Epoch 5/16
31/31 [==============================] - 1s 40ms/step - loss: 0.0279 - accuracy: 0.9878 - val_loss: 0.2707 - val_accuracy: 0.9515
Epoch 6/16
31/31 [==============================] - 1s 39ms/step - loss: 0.1170 - accuracy: 0.9715 - val_loss: 0.2441 - val_accuracy: 0.9818
Epoch 7/16
31/31 [==============================] - 1s 37ms/step - loss: 0.1623 - accuracy: 0.9736 - val_loss: 0.1538 - val_accuracy: 0.9879
Epoch 8/16
31

In [61]:
# results
testResults = model.predict(X_test)

print(confusion_matrix(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print("Cohen's Kappa: {}".format(cohen_kappa_score(y_test.argmax(axis=1), testResults.argmax(axis=1))))
print("Accuracy: ",accuracy_score(y_test.argmax(axis=1), testResults.argmax(axis=1)))

[[79  0]
 [ 7 79]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        79
           1       1.00      0.92      0.96        86

    accuracy                           0.96       165
   macro avg       0.96      0.96      0.96       165
weighted avg       0.96      0.96      0.96       165

Cohen's Kappa: 0.9153039524822175
Accuracy:  0.9575757575757575


```
[[79  0]
 [ 1 85]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        79
           1       1.00      0.99      0.99        86

    accuracy                           0.99       165
   macro avg       0.99      0.99      0.99       165
weighted avg       0.99      0.99      0.99       165

Cohen's Kappa: 0.9878631849944832
Accuracy:  0.9939393939393939
```

## Part 2

In [62]:
def load_img(indir, exclude = []):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir): 
            continue
        the_class = class_dir
        if the_class in exclude:
            continue
        print("Loading:",class_dir)
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('weather')
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels



Loading: cloudy
Loading: rain
Loading: shine
Loading: sunrise
loaded 1123  samples
classes {'shine', 'cloudy', 'rain', 'sunrise'}


In [63]:
# create end train/test dataset

labels_vec = LabelEncoder().fit_transform(org_labels)
one_hot_labels = keras.utils.to_categorical(labels_vec)
# org_samples, one_hot_labels = shuffle(org_samples, one_hot_labels)

X_train, X_test, y_train, y_test = train_test_split(org_samples, one_hot_labels, test_size=0.25, random_state=4244)

print(X_train.shape)
print(y_train.shape)

(842, 64, 64, 3)
(842, 4)


In [64]:
# create model
model2 = Sequential()
model2.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(256))
model2.add(Activation("relu"))
model2.add(Dense(4))
model2.add(Activation("softmax"))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [65]:
# train model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=24, batch_size=16)

Epoch 1/24
53/53 [==============================] - 4s 64ms/step - loss: 2.8168 - accuracy: 0.6758 - val_loss: 2.5046 - val_accuracy: 0.6690
Epoch 2/24
53/53 [==============================] - 4s 68ms/step - loss: 0.5177 - accuracy: 0.8420 - val_loss: 2.5716 - val_accuracy: 0.6121
Epoch 3/24
53/53 [==============================] - 3s 48ms/step - loss: 0.3571 - accuracy: 0.8729 - val_loss: 1.7585 - val_accuracy: 0.6940
Epoch 4/24
53/53 [==============================] - 3s 51ms/step - loss: 0.3031 - accuracy: 0.9002 - val_loss: 0.5674 - val_accuracy: 0.8327
Epoch 5/24
53/53 [==============================] - 3s 50ms/step - loss: 0.1230 - accuracy: 0.9667 - val_loss: 0.3901 - val_accuracy: 0.8719
Epoch 6/24
53/53 [==============================] - 3s 62ms/step - loss: 0.1007 - accuracy: 0.9620 - val_loss: 0.3192 - val_accuracy: 0.9181
Epoch 7/24
53/53 [==============================] - 3s 56ms/step - loss: 0.0948 - accuracy: 0.9667 - val_loss: 0.2625 - val_accuracy: 0.9181
Epoch 8/24
53

In [66]:
testResults = model2.predict(X_test)

print(confusion_matrix(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print("Cohen's Kappa: {}".format(cohen_kappa_score(y_test.argmax(axis=1), testResults.argmax(axis=1))))
print("Accuracy: ",accuracy_score(y_test.argmax(axis=1), testResults.argmax(axis=1)))

[[76  3  1  1]
 [ 8 43  0  0]
 [11  4 42  1]
 [ 0  1  0 90]]
              precision    recall  f1-score   support

           0       0.80      0.94      0.86        81
           1       0.84      0.84      0.84        51
           2       0.98      0.72      0.83        58
           3       0.98      0.99      0.98        91

    accuracy                           0.89       281
   macro avg       0.90      0.87      0.88       281
weighted avg       0.90      0.89      0.89       281

Cohen's Kappa: 0.8541497257738023
Accuracy:  0.8932384341637011


```
[[74  4  3  0]
 [ 7 44  0  0]
 [ 3  4 50  1]
 [ 0  0  1 90]]
              precision    recall  f1-score   support

           0       0.88      0.91      0.90        81
           1       0.85      0.86      0.85        51
           2       0.93      0.86      0.89        58
           3       0.99      0.99      0.99        91

    accuracy                           0.92       281
   macro avg       0.91      0.91      0.91       281
weighted avg       0.92      0.92      0.92       281

Cohen's Kappa: 0.8887454382703298
Accuracy:  0.9181494661921709
```